In [2]:
# === DAT490: Final rebuild for DP02–DP05 + combined + crime merge (openpyxl header-safe) ===
import pandas as pd
import numpy as np
import re
from pathlib import Path
from openpyxl import Workbook

BASE = Path(r"C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106")

DP_FILES = {
    "DP02": BASE / "ACSDP5Y2023.DP02-Data.csv",
    "DP03": BASE / "ACSDP5Y2023.DP03-Data.csv",
    "DP04": BASE / "ACSDP5Y2023.DP04-Data.csv",
    "DP05": BASE / "ACSDP5Y2023.DP05-Data.csv",
}
CRIME_FILE = BASE / "Crime_Reports.csv"

XLSX_OUT = {
    "DP02": BASE / "DP02_travis_county.xlsx",
    "DP03": BASE / "DP03_travis_county.xlsx",
    "DP04": BASE / "DP04_travis_county.xlsx",
    "DP05": BASE / "DP05_travis_county.xlsx",
}
COMBINED_OUT = BASE / "combined_acs_travis_county.csv"
DICT_XLSX   = BASE / "data_dictionary.xlsx"
MERGED_OUT  = BASE / "crime_with_acs_merge.csv"

TRAVIS_GEO_REGEX = re.compile(r"^1400000US48453\d{6}$")
VAR_RE = re.compile(r"^dp\d{2}_\d{4}(pe|pm|e|m)$", re.IGNORECASE)

# ---------- helpers ----------
def clean_dp(df: pd.DataFrame, table_id: str):
    """Rename to geo_id, name, and lowercase ACS codes. Return (df, row2_descriptions)."""
    df = df.copy()
    row1, row2 = [], []
    for c in df.columns:
        if c == "GEO_ID":
            row1.append("geo_id")
            row2.append("Census tract GEO_ID key (1400000US + state+county+tract)")
        elif c == "NAME":
            row1.append("name")
            row2.append("Geographic area name")
        else:
            lc = c.lower()
            row1.append(lc)
            if lc.endswith("pe"):
                row2.append(f"{table_id} {c} percentage estimate")
            elif lc.endswith("pm"):
                row2.append(f"{table_id} {c} percentage margin of error")
            elif lc.endswith("e"):
                row2.append(f"{table_id} {c} estimate")
            elif lc.endswith("m"):
                row2.append(f"{table_id} {c} margin of error")
            else:
                row2.append(f"{table_id} {c}")
    df.columns = row1
    return df, row2

def enforce_travis(df: pd.DataFrame) -> pd.DataFrame:
    """Keep only valid Travis County tract rows by GEO_ID pattern."""
    if "geo_id" not in df.columns:
        return df
    return df[df["geo_id"].fillna("").astype(str).str.match(TRAVIS_GEO_REGEX)]

def write_xlsx_two_header_openpyxl(path: Path, row1: list, row2: list, df: pd.DataFrame):
    """Write exact two header rows + data using openpyxl to avoid pandas header alignment issues."""
    wb = Workbook()
    ws = wb.active
    ws.title = "data"
    # Row 1: exact column names (must match df.columns)
    ws.append(list(row1))
    # Row 2: descriptions
    ws.append(list(row2))
    # Rows 3+: data rows (values only, in the same order as columns)
    for tup in df.itertuples(index=False, name=None):
        ws.append(list(tup))
    wb.save(path)

def derive_geo_id_from_cbg(series: pd.Series) -> pd.Series:
    cbg = series.astype(str).str.replace(r"\D", "", regex=True).str.zfill(10)
    tract11 = "48" + cbg.str[:3] + cbg.str[3:9]
    return "1400000US" + tract11

# ---------- load & clean from RAW (fresh) ----------
dp = {}
row2_map = {}

for dp_id, path in DP_FILES.items():
    raw = pd.read_csv(path, dtype=str, low_memory=False)
    df, r2 = clean_dp(raw, dp_id)
    df = enforce_travis(df)
    dp[dp_id] = df
    row2_map[dp_id] = r2

# ---------- write per-table Excel (openpyxl, exact headers) ----------
for dp_id in ["DP02","DP03","DP04","DP05"]:
    cols = list(dp[dp_id].columns)
    desc = row2_map[dp_id]
    # sanity: ensure same length
    assert len(cols) == len(desc), f"Row2 length mismatch in {dp_id}"
    write_xlsx_two_header_openpyxl(XLSX_OUT[dp_id], cols, desc, dp[dp_id])

# ---------- combine & save ----------
combined = (
    dp["DP02"]
    .merge(dp["DP03"], on=["geo_id","name"], how="outer")
    .merge(dp["DP04"], on=["geo_id","name"], how="outer")
    .merge(dp["DP05"], on=["geo_id","name"], how="outer")
)
combined.to_csv(COMBINED_OUT, index=False)

# ---------- data dictionary ----------
rows = []
for dp_id in ["DP02","DP03","DP04","DP05"]:
    row1 = list(dp[dp_id].columns)
    row2 = row2_map[dp_id]
    for col, dsc in zip(row1, row2):
        orig = col.upper() if VAR_RE.match(col) else col
        rows.append([dp_id, col, dsc, orig])
data_dict = pd.DataFrame(rows, columns=["source_table","column_name","short_description","original_code"])

# write dictionary via pandas (no special headers needed here)
try:
    with pd.ExcelWriter(DICT_XLSX, engine="xlsxwriter") as xw:
        data_dict.to_excel(xw, index=False, sheet_name="Data_Dictionary")
        pd.DataFrame({"Join & Cleaning Steps": [
            "1) Loaded DP02–DP05 (ACS 2023 5-Year, Travis County).",
            "2) Standardized headers: geo_id, name, and lowercase ACS codes; preserved original codes in dictionary.",
            "3) Built deterministic Row 2: E/M/PE/PM labeled (estimate / margin of error / percentage estimate / percentage margin of error).",
            "4) Enforced Travis GEO_ID pattern (1400000US48453xxxxxx).",
            "5) Wrote Excel via openpyxl (no implicit headers), then combined (outer join on geo_id, name).",
            "6) Derived crime geo_id from CBG and left-joined to ACS; kept merged file separate for integrity."
        ]}).to_excel(xw, index=False, sheet_name="Join_Steps")
        pd.DataFrame([["Crime Reports (Austin, TX)", "https://catalog.data.gov/dataset/crime-reports-bf2b7"],
                      ["ACS 5-Year Data Profiles (DP02–DP05)", "https://data.census.gov/"]],
                     columns=["Name","Link"]).to_excel(xw, index=False, sheet_name="Resources")
except ModuleNotFoundError:
    with pd.ExcelWriter(DICT_XLSX, engine="openpyxl") as xw:
        data_dict.to_excel(xw, index=False, sheet_name="Data_Dictionary")
        pd.DataFrame({"Join & Cleaning Steps": [
            "1) Loaded DP02–DP05 (ACS 2023 5-Year, Travis County).",
            "2) Standardized headers: geo_id, name, and lowercase ACS codes; preserved original codes in dictionary.",
            "3) Built deterministic Row 2: E/M/PE/PM labeled (estimate / margin of error / percentage estimate / percentage margin of error).",
            "4) Enforced Travis GEO_ID pattern (1400000US48453xxxxxx).",
            "5) Wrote Excel via openpyxl (no implicit headers), then combined (outer join on geo_id, name).",
            "6) Derived crime geo_id from CBG and left-joined to ACS; kept merged file separate for integrity."
        ]}).to_excel(xw, index=False, sheet_name="Join_Steps")
        pd.DataFrame([["Crime Reports (Austin, TX)", "https://catalog.data.gov/dataset/crime-reports-bf2b7"],
                      ["ACS 5-Year Data Profiles (DP02–DP05)", "https://data.census.gov/"]],
                     columns=["Name","Link"]).to_excel(xw, index=False, sheet_name="Resources")

# ---------- crime + ACS merge (chunked) ----------
acs_full = pd.read_csv(COMBINED_OUT, dtype=str, low_memory=False)

cbg_col = None
for c in pd.read_csv(CRIME_FILE, nrows=0).columns:
    if "census" in c.lower() and "block" in c.lower():
        cbg_col = c; break

first = True
for chunk in pd.read_csv(CRIME_FILE, dtype=str, low_memory=False, chunksize=100_000):
    if cbg_col:
        chunk["geo_id"] = derive_geo_id_from_cbg(chunk[cbg_col])
    else:
        chunk["geo_id"] = np.nan
    merged = chunk.merge(acs_full, on="geo_id", how="left")
    merged.to_csv(MERGED_OUT, index=False, mode="w" if first else "a", header=first)
    first = False

# ---------- mini checks ----------
def read_two_header_xlsx_openpyxl(path: Path):
    # for verification readback: use pandas header=None to capture both header rows
    x = pd.read_excel(path, sheet_name="data", header=None, dtype=str)
    row1 = [str(v) for v in x.iloc[0].tolist()]
    row2 = [("" if pd.isna(v) else str(v)) for v in x.iloc[1].tolist()]
    df   = x.iloc[2:].copy()
    df.columns = row1
    return row1, row2, df

print("=== OUTPUTS WRITTEN ===")
for k,v in XLSX_OUT.items():
    print(k, "->", v)
print("Combined ACS ->", COMBINED_OUT)
print("Dictionary   ->", DICT_XLSX)
print("Crime+ACS    ->", MERGED_OUT)

print("\n=== QUICK CHECKS ===")
for dp_id in ["DP02","DP03","DP04","DP05"]:
    r1, r2, df = read_two_header_xlsx_openpyxl(XLSX_OUT[dp_id])
    ok1 = {"geo_id","name"}.issubset(df.columns)
    raw = pd.read_csv(DP_FILES[dp_id], dtype=str, low_memory=False)
    raw_dp = [c for c in raw.columns if c not in ("GEO_ID","NAME")]
    df_dp  = [c for c in df.columns if c not in ("geo_id","name")]
    ok2 = set([c.lower() for c in raw_dp]) == set(df_dp)
    def desc_ok(row1,row2):
        VAR_RE2 = re.compile(r"^dp\d{2}_\d{4}(pe|pm|e|m)$")
        for h,d in zip(row1,row2):
            hl, dl = h.lower(), (d or "").lower()
            if hl in ("geo_id","name"):
                if not (("geo" in dl) or ("geographic" in dl)): return False
                continue
            if VAR_RE2.match(hl):
                if hl.endswith(("e","pe")) and "estimate" not in dl: return False
                if hl.endswith(("m","pm")) and "margin of error" not in dl: return False
        return True
    ok3 = desc_ok(r1, r2)
    ok4 = df["name"].fillna("").str.contains("Travis County; Texas", case=False).mean() > 0.95
    ok5 = df["geo_id"].fillna("").astype(str).str.match(TRAVIS_GEO_REGEX).mean() > 0.95
    print(f"{dp_id}: headers {ok1}, colsets {ok2}, desc {ok3}, travis {ok4}, georegex {ok5}")


=== OUTPUTS WRITTEN ===
DP02 -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\DP02_travis_county.xlsx
DP03 -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\DP03_travis_county.xlsx
DP04 -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\DP04_travis_county.xlsx
DP05 -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\DP05_travis_county.xlsx
Combined ACS -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\combined_acs_travis_county.csv
Dictionary   -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\data_dictionary.xlsx
Crime+ACS    -> C:\Users\nicol\OneDrive\DAT490\Travis_County_ACS_2025-10-26T122106\crime_with_acs_merge.csv

=== QUICK CHECKS ===
DP02: headers True, colsets True, desc True, travis True, georegex True
DP03: headers True, colsets True, desc True, travis True, georegex True
DP04: headers True, colsets True, desc True, travis True, georegex True
DP05: headers Tr